In [1]:
from bs4 import BeautifulSoup as bs
import requests

In [2]:
url = 'https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&year1=2016&year2=2022&minmsrpsel=0&maxmsrpsel=0&city=0&hwy=0&comb=0&YearSel=2016-2022&make=&mclass=&vfuel=&vtype=&trany=&drive=&cyl=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=50&pageno=1&tabView=0'

In [3]:
response = requests.get(url)

In [4]:
response

<Response [200]>

In [5]:
soup = bs(response.text, 'html.parser')

In [ ]:
soup

In [6]:
## body
tb = soup.find('tbody')

In [7]:
## all the rows
rows = tb.find_all('tr', class_ = 'ymm-row')

In [ ]:
rows

In [ ]:
## car names
for name in rows:
  carfind = name.find('a')
  car = carfind.text
  print(car)

In [ ]:
## fuel & manual type
for name in rows:
  drivefind = name.find('span')
  drive = drivefind.text
  print(drive)

In [ ]:
## combined miles per galon
mpg_row = tb.find_all('td', class_ = 'mpg-comb')

for name in mpg_row:
  mpg = name.get_text()
  print(mpg)

In [ ]:
## annual fuel cost
fuel_row = tb.find_all('td', class_ = 'ann-fuel-cost')

for name in fuel_row:
  fuel = name.get_text()
  fuel = fuel[1:]
  print(fuel)

In [21]:
import pandas as pd

In [ ]:
## Inserting the first page into a dataframe
car_data = pd.DataFrame(columns=['Car Name', 'Manual/Motor', 'MPG', 'Annual Fuel Cost'])
for name, mpg_name, fuel_name in zip(rows, mpg_row, fuel_row):
    carfind = name.find('a')
    car = carfind.text

    drivefind = name.find('span')
    drive = drivefind.text

    mpg = mpg_name.get_text()

    fuel = fuel_name.get_text()[1:]

# Appending the data to the DataFrame
    car_data = car_data.append({'Car Name': car, 'Manual/Motor': drive, 'MPG': mpg, 'Annual Fuel Cost': fuel}, ignore_index=True)

In [ ]:
car_data

In [ ]:
## Bring all the pages into 1 dataframe
car_infos = pd.DataFrame(columns=['Car Name', 'Manual/Motor', 'MPG', 'Annual Fuel Cost'])
for i in range(1,183):
  url = f'https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&year1=2016&year2=2022&minmsrpsel=0&maxmsrpsel=0&city=0&hwy=0&comb=0&YearSel=2016-2022&make=&mclass=&vfuel=&vtype=&trany=&drive=&cyl=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=50&pageno={i}&tabView=0'
  response = requests.get(url)
  soup = bs(response.text, 'html.parser')
  tb = soup.find('tbody')
  rows = tb.find_all('tr', class_ = 'ymm-row')
  mpg_row = tb.find_all('td', class_ = 'mpg-comb')
  fuel_row = tb.find_all('td', class_ = 'ann-fuel-cost')

  for name, mpg_name, fuel_name in zip(rows, mpg_row, fuel_row):
      carfind = name.find('a')
      car = carfind.text

      drivefind = name.find('span')
      drive = drivefind.text

      mpg = mpg_name.get_text()

      fuel = fuel_name.get_text().strip()

# Append the data to the DataFrame
      car_infos = car_infos.append({'Car Name': car, 'Manual/Motor': drive, 'MPG': mpg, 'Annual Fuel Cost': fuel}, ignore_index=True)


In [27]:
car_infos

,Car Name,Manual/Motor,MPG,Annual Fuel Cost
0,2021 Tesla Model 3 Standard Range Plus RWD,"Automatic (A1), Electricity",142,$500
1,2020 Tesla Model 3 Standard Range Plus,"Automatic (A1), Electricity",141,$500
2,2019 Hyundai Ioniq Electric,"Automatic (A1), Electricity",136,$500
3,2018 Hyundai Ioniq Electric,"Automatic (A1), Electricity",136,$500
4,2017 Hyundai Ioniq Electric,"Automatic (A1), Electricity",136,$500
...,...,...,...,...
9094,2021 Lamborghini Aventador Sian Roadster,"6.5 L, 12 cyl, Automatic (AM-S7), Premium Gaso...",10,"$6,650"
9095,2020 Lamborghini Aventador Sian Roadster,"6.5 L, 12 cyl, Automatic (AM-S7), Premium Gaso...",10,"$6,650"
9096,2021 Bugatti Chiron Pur Sport,"8.0 L, 16 cyl, Automatic (AM-S7), Turbo, Premi...",10,"$6,650"
9097,2022 Bugatti Chiron Pur Sport,"8.0 L, 16 cyl, Automatic (AM-S7), Turbo, Premi...",9,"$7,400"


In [ ]:
car_infos.to_csv('cars.csv')